In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install streamlit
!pip install pandas
!pip install matplotlib
!pip install seaborn
!pip install scikit-learn
!pip install imbalanced-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
%%writefile app.py

import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

st.title("Disease Predictor")
selection = st.radio("Choose the Disease:", ["Diabetes", "Stroke", "Framingham Heart Disease"])

def plot_evaluation_metrics(y_test, y_pred, model_name):
    """Plot evaluation metrics for a given model and return the Matplotlib figure."""
    acc = accuracy_score(y_test, y_pred)
    class_report = classification_report(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)

    # Plotting the confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
    plt.title(f'{model_name} - Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')

    # Capture the Matplotlib figure
    fig = plt.gcf()

    # Display the plot in Streamlit
    st.pyplot(fig)

    st.write(f"Accuracy: {acc:.2f}")
    st.write("Classification Report:\n", class_report)


if selection == "Diabetes":
    st.title('Diabetes Prediction')
    file_path = '/content/drive/My Drive/diabetes.csv'
    df = pd.read_csv(file_path)

    X = df.drop('Outcome', axis=1)
    y = df['Outcome']
    cols_with_zeros = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
    X[cols_with_zeros] = X[cols_with_zeros].replace(0, pd.NA)
    X.fillna(X.mean(), inplace=True)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    model = LogisticRegression()
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)

    accuracy = accuracy_score(y_test, y_pred)
    st.write(f"Accuracy: {accuracy:.2f}")

    st.write("Enter the details:")
    col1, col2 = st.columns(2)
    with col1:
        preg = st.number_input('Pregnancies')
        glucose = st.number_input('Glucose')
        bp = st.number_input('BloodPressure')
        skin = st.number_input('SkinThickness')
    with col2:
        insulin = st.number_input('Insulin')
        bmi = st.number_input('BMI')
        dpf = st.number_input('DiabetesPedigreeFunction')
        age = st.number_input('Age')

    new_data = [[preg, glucose, bp, skin, insulin, bmi, dpf, age]]
    new_data_scaled = scaler.transform(new_data)
    if st.button('Predict Diabetes'):
        predictions = model.predict(new_data_scaled)
        st.write(f"Prediction: {'Positive' if predictions[0] == 1 else 'Negative'}")

        # Show evaluation metrics
        plot_evaluation_metrics(y_test, y_pred, "Diabetes_Model")

elif selection == "Stroke":
    st.title('Stroke Prediction')
    df = pd.read_csv('/content/drive/My Drive/healthcare-dataset-stroke-data.csv')
    df['bmi'].fillna(df['bmi'].mean(), inplace=True)
    df['smoking_status'].fillna('unknown', inplace=True)

    gender_encoder = LabelEncoder().fit(df['gender'])
    marriage_encoder = LabelEncoder().fit(df['ever_married'])
    work_type_encoder = LabelEncoder().fit(df['work_type'])
    residence_encoder = LabelEncoder().fit(df['Residence_type'])
    smoking_encoder = LabelEncoder().fit(df['smoking_status'])

    df['gender'] = gender_encoder.transform(df['gender'])
    df['ever_married'] = marriage_encoder.transform(df['ever_married'])
    df['work_type'] = work_type_encoder.transform(df['work_type'])
    df['Residence_type'] = residence_encoder.transform(df['Residence_type'])
    df['smoking_status'] = smoking_encoder.transform(df['smoking_status'])

    scaler = StandardScaler()
    df[['age', 'avg_glucose_level', 'bmi']] = scaler.fit_transform(df[['age', 'avg_glucose_level', 'bmi']])

    X = df.drop(columns=['id', 'stroke'])
    y = df['stroke']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    st.write(f"Accuracy: {accuracy:.2f}")

    st.write("Enter the details:")
    col1, col2 = st.columns(2)
    with col1:
        gender = st.selectbox('Gender', gender_encoder.classes_)
        age = st.number_input('Age')
        hypertension = st.selectbox('Hypertension', [0, 1])
        heart_disease = st.selectbox('Heart Disease', [0, 1])
        ever_married = st.selectbox('Ever Married', marriage_encoder.classes_)
    with col2:
        work_type = st.selectbox('Work Type', work_type_encoder.classes_)
        residence_type = st.selectbox('Residence Type', residence_encoder.classes_)
        avg_glucose_level = st.number_input('Average Glucose Level')
        bmi = st.number_input('BMI')
        smoking_status = st.selectbox('Smoking Status', smoking_encoder.classes_)

    new_data = [[gender_encoder.transform([gender])[0], age, hypertension, heart_disease, marriage_encoder.transform([ever_married])[0],
                 work_type_encoder.transform([work_type])[0], residence_encoder.transform([residence_type])[0], avg_glucose_level, bmi,
                 smoking_encoder.transform([smoking_status])[0]]]
    if st.button('Predict Stroke'):
        predictions = model.predict(new_data)
        st.write(f"Prediction: {'Positive' if predictions[0] == 1 else 'Negative'}")

        # Show evaluation metrics
        plot_evaluation_metrics(y_test, y_pred, "Stroke_Model")

elif selection == "Framingham Heart Disease":
    st.title('Framingham Heart Disease Prediction')
    df = pd.read_csv('/content/drive/My Drive/framingham.csv')
    df.fillna(df.mean(), inplace=True)
    X = df.drop('TenYearCHD', axis=1)
    y = df['TenYearCHD']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    model = RandomForestClassifier()
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)

    accuracy = accuracy_score(y_test, y_pred)
    st.write(f"Accuracy: {accuracy:.2f}")

    st.write("Enter the details:")
    col1, col2 = st.columns(2)
    with col1:
        male = st.selectbox('Gender', ['Male', 'Female'])
        age = st.number_input('Age')
        education = st.selectbox('Education', [1, 2, 3, 4])
        currentSmoker = st.selectbox('Current Smoker', [0, 1])
        cigsPerDay = st.number_input('Cigarettes per Day')
        BPMeds = st.selectbox('Blood Pressure Medication', [0, 1])
    with col2:
        prevalentStroke = st.selectbox('Prevalent Stroke', [0, 1])
        prevalentHyp = st.selectbox('Prevalent Hypertension', [0, 1])
        diabetes = st.selectbox('Diabetes', [0, 1])
        totChol = st.number_input('Total Cholesterol')
        sysBP = st.number_input('Systolic Blood Pressure')
        diaBP = st.number_input('Diastolic Blood Pressure')
        BMI = st.number_input('BMI')
        heartRate = st.number_input('Heart Rate')
        glucose = st.number_input('Glucose')

    new_data = [[1 if male == 'Male' else 0, age, education, currentSmoker, cigsPerDay, BPMeds, prevalentStroke, prevalentHyp, diabetes, totChol, sysBP, diaBP, BMI, heartRate, glucose]]
    new_data_scaled = scaler.transform(new_data)
    if st.button('Predict Heart Disease'):
        predictions = model.predict(new_data_scaled)
        st.write(f"Prediction: {'Positive' if predictions[0] == 1 else 'Negative'}")

        # Show evaluation metrics
        plot_evaluation_metrics(y_test, y_pred, "Framingham_Model")


Writing app.py


In [ ]:
! wget -q -O - ipv4.icanhazip.com

34.67.177.78


In [ ]:
! streamlit run app.py & npx localtunnel --port 8501





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.67.177.78:8501

Need to install the following packages:
  localtunnel@2.0.2
Ok to proceed? (y) 